Python code for generating marginal position distribution for a CRTP starting at the origin. We use isotropic initial conditions. Let the unit vector pointing in the direction of motion be [$\cos\theta$,$\sin\theta$]. Let the states $\{0,1,2,3\}$ correspond to the orientations $\{0,\pi/2,\pi,3\pi/2\}$

In [ ]:
import random
import numpy as np
import csv

# Function returns x component of unit vector pointing in direction of motion.

def f_x(state):
	if state == 0: trial = 1
	if state == 1 or state == 3: trial = 0
	if state == 2: trial = -1
	return trial

# Here we assume that the rate of flipping (\Gamma_2) is 0. The first entry of the--
# --corresponds to rate of tumbling left (\Gamma_1) and the second entry corresponds--
# --to rate of tumbling right (\Gamma_3).

gamma = [0.4, 1.0]
n = 10**4 # Number of trial trajectories.
v = 1.0   # Constant speed of CRTP.

with open ('trail0', 'w') as f:    # We write the data generated to a CSV file.
    writer = csv.writer(f)

    for k in range(n):
        state = random.randint(0,3) # Randomly choose one of the four possible-- 
                                    # --states as the initial one.
        t, t_f, x = 0.0, 5.0, 0.0   # Time of starting and ending trajectory--
                                    # --and the starting position of particle. 
        while t <= t_f:

            # The time for subsequent tumble to be left or right is exponentially-- 
            # --distributed with different average rates.
            # We draw a random number from each distribution.
            # The next move, left or right, corresponds to the distribution from--
            # --which the lesser of the two numbers is drawn from.
            # The lesser of the two numbers is the time spent in current state.
            # See The First Reaction method

            wait = [-np.log(random.random())/gamma[i] for i in range(2)]
            tau = min(wait)
            diff = t + tau

            # Proceed if the sum of current time and time to be spent in current--
            # --state before the next tumble is less than ending time of trajectory.

            if diff < t_f:
                x = x + f_x(state)*v*tau # The value of x coordinate is updated.
                j = wait.index(tau)      # Check whether the the lesser of the-- 
                                         # --random numbers drawn corresponds to-- 
                                         # --left or right tumble and update state.
                
                if j == 0:
                    if state == 3: state = 0
                    else: state = state + 1
                
                if j == 1:
                    if state == 0: state = 3
                    else: state = state - 1
            
            # If the sum of current time and time to be spent in current state--
            # --before next tumble is more than ending time of trajectory, update-- 
            # --the position only according to time difference of end time of--
            # --trajectory and current time.
            
            if diff >= t_f: 
                delta = t_f - t
                x = x + f_x(state)*v*delta
                writer.writerow([x])        # Write the final position to a file-- 
                                            # --to generate histogram for the marginal 
                                            # position distribution.
                break                       # Exit and repeat for another trajectory.

            t = t + tau  

Python code for generating the data for the plot of MFPT vs $x$ where $x$ is the starting position between two infinte vertical boundaries at $x = \pm L$.

In [ ]:
# The first entry of list corresponds to rate of tumbling left (\Gamma_1),--
# --the second entry corresponds to rate of flipping (\Gamma_2) and the third entry-- 
# --corresponds to rate of tumbling right (\Gamma_3).

gamma = [0.4, 0.2, 1.0]

v = 1.0        # Constant speed of CRTP.
n = 10**4      # Number of trial trajectories for one starting position.
t_f = 10000.0  # The position of the CRTP is updated until the target is found,-- 
               # --but computationally it is a good idea to put an upper limit--
               # --on time for one trajectory.
MFPT = []      # List stores the MFPT for one starting position.
X_list = np.linspace(-10.0,10.0,21) # List of equally spaced starting positions.

for m in X_list:

    First_Passage = []   # List to store FPT for one trajectory for a specific--
                         # --starting position.

    for i in range(n):

        state = 2        # Choose an initial state.
        t, x = 0.0, m    # Set the initial time and starting position.

        while t <= t_f:

            # The algorithm for updation of position and state remains the same.

            wait = [-np.log(random.random())/gamma[k] for k in range(3)]
            tau = min(wait)

            if state == 0: f_x = 1
            if state == 1: f_x = 0
            if state == 2: f_x = -1 
            if state == 3: f_x = 0

            # If absolute value of position after updation is greater than L (here--
            # --10.0) then FPT equals time before updation added to the time taken--
            # --to reach the boundary from the position prior to updation.

            if abs(x + f_x*v*tau) >= 10.0: 
                First_Passage.append(t + (10.0 - abs(x))/v)
                break

            # Proceed otherwise.

            x = x + f_x*v*tau

            j = wait.index(tau)

            if j == 0:
                if state == 3: state = 0
                else: state = state + 1

            if j == 1: state = (state+2)%4

            if j == 2:
                if state == 0: state = 3
                else: state = state - 1

            t = t + tau  

    MFPT.append(np.mean(First_Passage))


# Write the data for starting position and the numerically evaluated MFPT to a--
# --CSV file.

data = [[X_list[i],MFPT[i]] for i in range(len(MFPT))]

with open ('trail', 'w', newline='') as f:
    writer = csv.writer(f)
    for i in data: writer.writerow(i)